In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import pandas as pd

In [3]:
DEBUG_PROMPT = False

In [5]:
token = spotipy.util.prompt_for_user_token('Julien Salomon',
                           client_id='3f0268c00c604e70aace4982367723c7',
                           client_secret='c9a4a4f2699642c795cb82595c2c8035',
                            redirect_uri='http://localhost:8888')

In [6]:
spotify = spotipy.Spotify(auth=token)
spotify.trace = False # turn on tracing
spotify.trace_out = False # turn on trace out

In [7]:
def print_list(lis, tab=0):
    tabs = ''
    for i in range(tab):
        tabs += '\t'
    print(tabs + 'list_length: ' + str(len(lis)))
    for val in lis:
        if isinstance(val, dict):
            print_dict(val, tab=tab)
        elif isinstance(val, list):
            if len(val) <= 50 or DEBUG_PROMPT:
                print_list(val, tab=tab + 1)
        else:
            print(tabs + val)
            
def print_dict(dico, tab=0):
    tabs = ''
    for i in range(tab):
        tabs += '\t'
    for item, value in dico.items():
        if isinstance(value, dict):
            print(tabs + item + ':')
            print_dict(value, tab=tab+1)
        elif isinstance(value, list):
            print(tabs + item + ':')
            print_list(value, tab=tab + 1)
        else:
            print(tabs + item + ' : ' + str(value))

In [8]:
def print_result(res, info=None):
    if info:
        print(info)
    if isinstance(res, dict):
        print_dict(res)
    elif isinstance(res, list):
        print_list(res)
    else:
        print(res)

In [9]:
def get_artist(artist_name):
    results = spotify.search(artist_name, type='artist')['artists']['items']
    results.sort(key=lambda artist: artist['popularity'], reverse=True)
    top_result = results[0]
    if DEBUG_PROMPT:
        print_result(top_result, info='Artist')
    return top_result

In [10]:
def get_albums_tracks(albums):
    all_tracks = []
    for uri in albums:
        tracks = spotify.album_tracks(uri)['items']
        for track in tracks:
            track['album_uri'] = uri
        all_tracks += tracks
    
    return pd.DataFrame(all_tracks)

In [11]:
def get_audio_features(tracks):
    features_list = [spotify.audio_features(track)[0] for track in tracks]
    features = pd.DataFrame(features_list)
    features['uri_track'] = features['uri']
    features = features.drop('duration_ms', axis=1).drop('uri', axis=1)
    if DEBUG_PROMPT:
        print(features.head())
    return features

In [12]:
def get_audio_analysis(tracks):
    analysis_list = [spotify.audio_analysis(track) for track in tracks]
    analysis = pd.DataFrame(analysis_list)
    analysis['uri'] = tracks
    if DEBUG_PROMPT:
        print(analysis.head())
    return analysis

In [13]:
mj_albums = spotify.artist_albums(get_artist('Michael Jackson')['uri'])['items']
mj_df = pd.DataFrame.from_dict(mj_albums)
mj_df.head()

,album_group,album_type,artists,available_markets,external_urls,href,id,images,name,release_date,release_date_precision,total_tracks,type,uri
0,album,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",{'spotify': 'https://open.spotify.com/album/2X...,https://api.spotify.com/v1/albums/2X8UOIkZQdcz...,2X8UOIkZQdcz2Hi5Ynt2uk,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Scream,2017-09-27,day,14,album,spotify:album:2X8UOIkZQdcz2Hi5Ynt2uk
1,album,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",{'spotify': 'https://open.spotify.com/album/7p...,https://api.spotify.com/v1/albums/7pomP86PUhoJ...,7pomP86PUhoJpY3fsC0WDQ,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",XSCAPE,2014-05-09,day,17,album,spotify:album:7pomP86PUhoJpY3fsC0WDQ
2,album,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",{'spotify': 'https://open.spotify.com/album/54...,https://api.spotify.com/v1/albums/54WFfZzds26N...,54WFfZzds26NrwcmStfamd,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",XSCAPE - Track by Track Commentary,2014-05-09,day,8,album,spotify:album:54WFfZzds26NrwcmStfamd
3,album,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",{'spotify': 'https://open.spotify.com/album/0y...,https://api.spotify.com/v1/albums/0yYWeLqonFk1...,0yYWeLqonFk11pmb0RiMbP,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Michael,2010-12-14,day,10,album,spotify:album:0yYWeLqonFk11pmb0RiMbP
4,album,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",{'spotify': 'https://open.spotify.com/album/52...,https://api.spotify.com/v1/albums/52E4RP7XDzal...,52E4RP7XDzalpIrOgSTgiQ,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Invincible,2001-10-29,day,16,album,spotify:album:52E4RP7XDzalpIrOgSTgiQ


In [14]:
mj_tracks_df = get_albums_tracks(mj_df.uri.unique())
mj_tracks_df.head()

,artists,available_markets,disc_number,duration_ms,explicit,external_urls,href,id,is_local,name,preview_url,track_number,type,uri,album_uri
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,343413,False,{'spotify': 'https://open.spotify.com/track/2B...,https://api.spotify.com/v1/tracks/2BU4dK1fu1CF...,2BU4dK1fu1CFLjTBz7ju7Q,False,This Place Hotel (a.k.a. Heartbreak Hotel),https://p.scdn.co/mp3-preview/f17429c3426b249c...,1,track,spotify:track:2BU4dK1fu1CFLjTBz7ju7Q,spotify:album:2X8UOIkZQdcz2Hi5Ynt2uk
1,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,358053,False,{'spotify': 'https://open.spotify.com/track/7a...,https://api.spotify.com/v1/tracks/7azo4rpSUh8n...,7azo4rpSUh8nXgtonC6Pkq,False,Thriller,https://p.scdn.co/mp3-preview/9f288fba4d3380b1...,2,track,spotify:track:7azo4rpSUh8nXgtonC6Pkq,spotify:album:2X8UOIkZQdcz2Hi5Ynt2uk
2,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,253373,False,{'spotify': 'https://open.spotify.com/track/0Q...,https://api.spotify.com/v1/tracks/0QgsvILuE9VQ...,0QgsvILuE9VQiAVsLStD3W,False,Blood on the Dance Floor,https://p.scdn.co/mp3-preview/6ef56a67f7da9122...,3,track,spotify:track:0QgsvILuE9VQiAVsLStD3W,spotify:album:2X8UOIkZQdcz2Hi5Ynt2uk
3,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,237760,False,{'spotify': 'https://open.spotify.com/track/6A...,https://api.spotify.com/v1/tracks/6A4Jc8npNo79...,6A4Jc8npNo79BOgsrPptLA,False,Somebody's Watching Me - Single Version,https://p.scdn.co/mp3-preview/c1245127fedad1ee...,4,track,spotify:track:6A4Jc8npNo79BOgsrPptLA,spotify:album:2X8UOIkZQdcz2Hi5Ynt2uk
4,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,280946,False,{'spotify': 'https://open.spotify.com/track/4F...,https://api.spotify.com/v1/tracks/4FC7GMnGwYeA...,4FC7GMnGwYeAjuvC83Srbs,False,Dirty Diana - 2012 Remaster,https://p.scdn.co/mp3-preview/532affb8368ee2a5...,5,track,spotify:track:4FC7GMnGwYeAjuvC83Srbs,spotify:album:2X8UOIkZQdcz2Hi5Ynt2uk


In [15]:
mj_tracks_df.iloc[0].external_urls

{'spotify': 'https://open.spotify.com/track/2BU4dK1fu1CFLjTBz7ju7Q'}

In [18]:
mj_df['album_uri'] = mj_df.uri
mj_df = mj_tracks_df.join(mj_df.set_index('album_uri'), lsuffix='_track', rsuffix='_album', on='album_uri')
if DEBUG_PROMPT:
    print(mj_df.columns)

In [ ]:
#Add tracks for non-MJ albums

In [19]:
track_uris = mj_df.uri_track.unique()

In [20]:
features = get_audio_features(track_uris)

retrying after...6secs
retrying after...6secs


In [21]:
mj_df = mj_df.join(features.set_index('uri_track'), on='uri_track')

In [22]:
mj_df.head()

,artists_track,available_markets_track,disc_number,duration_ms,explicit,external_urls_track,href_track,id_track,is_local,name_track,...,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,time_signature
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,343413,False,{'spotify': 'https://open.spotify.com/track/2B...,https://api.spotify.com/v1/tracks/2BU4dK1fu1CF...,2BU4dK1fu1CFLjTBz7ju7Q,False,This Place Hotel (a.k.a. Heartbreak Hotel),...,0.2430,0.000142,0.1170,0.582,97.005,audio_features,2BU4dK1fu1CFLjTBz7ju7Q,https://api.spotify.com/v1/tracks/2BU4dK1fu1CF...,https://api.spotify.com/v1/audio-analysis/2BU4...,4
1,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,358053,False,{'spotify': 'https://open.spotify.com/track/7a...,https://api.spotify.com/v1/tracks/7azo4rpSUh8n...,7azo4rpSUh8nXgtonC6Pkq,False,Thriller,...,0.0816,0.000108,0.8470,0.721,118.421,audio_features,7azo4rpSUh8nXgtonC6Pkq,https://api.spotify.com/v1/tracks/7azo4rpSUh8n...,https://api.spotify.com/v1/audio-analysis/7azo...,4
2,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,253373,False,{'spotify': 'https://open.spotify.com/track/0Q...,https://api.spotify.com/v1/tracks/0QgsvILuE9VQ...,0QgsvILuE9VQiAVsLStD3W,False,Blood on the Dance Floor,...,0.0148,0.009780,0.0623,0.520,108.980,audio_features,0QgsvILuE9VQiAVsLStD3W,https://api.spotify.com/v1/tracks/0QgsvILuE9VQ...,https://api.spotify.com/v1/audio-analysis/0Qgs...,4
3,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,237760,False,{'spotify': 'https://open.spotify.com/track/6A...,https://api.spotify.com/v1/tracks/6A4Jc8npNo79...,6A4Jc8npNo79BOgsrPptLA,False,Somebody's Watching Me - Single Version,...,0.0409,0.000081,0.1320,0.804,124.091,audio_features,6A4Jc8npNo79BOgsrPptLA,https://api.spotify.com/v1/tracks/6A4Jc8npNo79...,https://api.spotify.com/v1/audio-analysis/6A4J...,4
4,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,280946,False,{'spotify': 'https://open.spotify.com/track/4F...,https://api.spotify.com/v1/tracks/4FC7GMnGwYeA...,4FC7GMnGwYeAjuvC83Srbs,False,Dirty Diana - 2012 Remaster,...,0.1780,0.000401,0.3240,0.331,131.424,audio_features,4FC7GMnGwYeAjuvC83Srbs,https://api.spotify.com/v1/tracks/4FC7GMnGwYeA...,https://api.spotify.com/v1/audio-analysis/4FC7...,4


In [24]:
mj_df['featuring'] = mj_df.artists_track.map(lambda l: len(l) > 1)
mj_df['artists_track_uri'] = mj_df.artists_track.map(lambda artist_list: [artist['uri'] for artist in artist_list])
mj_df['artists_track'] = mj_df.artists_track.map(lambda artist_list: [artist['name'] for artist in artist_list])

mj_df = mj_df[['name_track', 'duration_ms', 'explicit', 'artists_track', 'artists_track_uri', 'featuring', 'track_number', 'disc_number', 'name_album', 'artists_album', 'total_tracks', 'uri_track', 'uri_album', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]

TypeError: string indices must be integers

In [70]:
for x in list(mj_df.name_track.unique()):
    if '(' in x:
        print(x)

This Place Hotel (a.k.a. Heartbreak Hotel)
Blood On The Dance Floor X Dangerous (The White Panda Mash-Up)
Love Never Felt So Good (Original Version)
Chicago (Original Version)
Loving You (Original Version)
A Place With No Name (Original Version)
Slave to the Rhythm (Original Version)
Do You Know Where Your Children Are (Original Version)
Blue Gangsta (Original Version)
Xscape (Original Version)
(I Like) The Way You Love Me
(I Can't Make It) Another Day
The Girl Is Mine (with Paul McCartney)
Childhood (Theme from "Free Willy 2")
I Just Can't Stop Loving You (feat. Siedah Garrett) - 2012 Remaster
Song Groove (a.k.a. Abortion Papers)
Todo Mi Amor Eres Tu (I Just Can't Stop Loving You)
Je Ne Veux Pas La Fin De Nous (I Just Can't Stop Loving You) - French Version
Bad (feat. Pitbull) - Remix By AfroJack Featuring Pitbull-DJ Buddha Edit
Speed Demon (Remix by Nero)
P.Y.T. (Pretty Young Thing)
The Girl Is Mine 2008 with will.i.am (with will.i.am) - Thriller 25th Anniversary Remix
P.Y.T. (Pretty

In [414]:
mj_df.to_pickle('mj_df.pickle')

In [52]:
mj_df.head()
mj_df.columns

Index(['name_track', 'duration_ms', 'explicit', 'artists_track',
       'artists_track_uri', 'featuring', 'track_number', 'disc_number',
       'name_album', 'artists_album', 'total_tracks', 'uri_track', 'uri_album',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature'],
      dtype='object')

In [33]:
analysis_df = pd.DataFrame(get_audio_analysis(track_uris))
analysis_df.head()

,meta,track,bars,beats,tatums,sections,segments,uri
0,"{'analyzer_version': '4.0.0', 'platform': 'Lin...","{'num_samples': 7572264, 'duration': 343.41333...","[{'start': 2.77178, 'duration': 2.39216, 'conf...","[{'start': 0.94698, 'duration': 0.62592, 'conf...","[{'start': 0.94698, 'duration': 0.31296, 'conf...","[{'start': 0.0, 'duration': 15.08662, 'confide...","[{'start': 0.0, 'duration': 0.21605, 'confiden...",spotify:track:2BU4dK1fu1CFLjTBz7ju7Q
1,"{'analyzer_version': '4.0.0', 'platform': 'Lin...","{'num_samples': 7895076, 'duration': 358.05333...","[{'start': 0.44776, 'duration': 1.97357, 'conf...","[{'start': 0.44776, 'duration': 0.48946, 'conf...","[{'start': 0.44776, 'duration': 0.24473, 'conf...","[{'start': 0.0, 'duration': 18.33347, 'confide...","[{'start': 0.0, 'duration': 0.18485, 'confiden...",spotify:track:7azo4rpSUh8nXgtonC6Pkq
2,"{'analyzer_version': '4.0.0', 'platform': 'Lin...","{'num_samples': 5586882, 'duration': 253.37333...","[{'start': 0.12478, 'duration': 2.20566, 'conf...","[{'start': 0.12478, 'duration': 0.55169, 'conf...","[{'start': 0.12478, 'duration': 0.27585, 'conf...","[{'start': 0.0, 'duration': 18.8512, 'confiden...","[{'start': 0.0, 'duration': 0.10431, 'confiden...",spotify:track:0QgsvILuE9VQiAVsLStD3W
3,"{'analyzer_version': '4.0.0', 'platform': 'Lin...","{'num_samples': 5242608, 'duration': 237.76, '...","[{'start': 1.0966, 'duration': 1.93872, 'confi...","[{'start': 0.12311, 'duration': 0.48341, 'conf...","[{'start': 0.12311, 'duration': 0.24171, 'conf...","[{'start': 0.0, 'duration': 4.00048, 'confiden...","[{'start': 0.0, 'duration': 0.09279, 'confiden...",spotify:track:6A4Jc8npNo79BOgsrPptLA
4,"{'analyzer_version': '4.0.0', 'platform': 'Lin...","{'num_samples': 6194874, 'duration': 280.94667...","[{'start': 1.45902, 'duration': 1.84623, 'conf...","[{'start': 0.12966, 'duration': 0.4463, 'confi...","[{'start': 0.12966, 'duration': 0.22315, 'conf...","[{'start': 0.0, 'duration': 12.38671, 'confide...","[{'start': 0.0, 'duration': 0.09465, 'confiden...",spotify:track:4FC7GMnGwYeAjuvC83Srbs


In [49]:
analysis_df.to_pickle('mj_analysis.pickle')

In [63]:
analysis_df.iloc[0].track.keys()

dict_keys(['num_samples', 'duration', 'sample_md5', 'offset_seconds', 'window_seconds', 'analysis_sample_rate', 'analysis_channels', 'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'codestring', 'code_version', 'echoprintstring', 'echoprint_version', 'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version'])

In [3]:
pd.read_pickle('michael_jackson_tracks.pickle')

,name_track,duration_ms,explicit,artists_track,artists_track_uri,featuring,track_number,disc_number,name_album,artists_album,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,This Place Hotel (a.k.a. Heartbreak Hotel),343413,False,[The Jacksons],[spotify:artist:2yrbLiuBmc9j81lTX3XUuI],False,1,1,Scream,[{'external_urls': {'spotify': 'https://open.s...,...,4,-4.085,0,0.1030,0.2430,0.000142,0.1170,0.582,97.005,4
1,Thriller,358053,False,[Michael Jackson],[spotify:artist:3fMbdgg4jU18AjLCKBhRSm],False,2,1,Scream,[{'external_urls': {'spotify': 'https://open.s...,...,11,-3.725,1,0.0738,0.0816,0.000108,0.8470,0.721,118.421,4
2,Blood on the Dance Floor,253373,False,[Michael Jackson],[spotify:artist:3fMbdgg4jU18AjLCKBhRSm],False,3,1,Scream,[{'external_urls': {'spotify': 'https://open.s...,...,1,-2.767,1,0.0764,0.0148,0.009780,0.0623,0.520,108.980,4
3,Somebody's Watching Me - Single Version,237760,False,[Rockwell],[spotify:artist:3xs0LEzcPXtgNfMNcHzLIP],False,4,1,Scream,[{'external_urls': {'spotify': 'https://open.s...,...,1,-4.128,0,0.0324,0.0409,0.000081,0.1320,0.804,124.091,4
4,Dirty Diana - 2012 Remaster,280946,False,[Michael Jackson],[spotify:artist:3fMbdgg4jU18AjLCKBhRSm],False,5,1,Scream,[{'external_urls': {'spotify': 'https://open.s...,...,3,-4.525,1,0.0437,0.1780,0.000401,0.3240,0.331,131.424,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Doggin' Around,172866,False,[Michael Jackson],[spotify:artist:3fMbdgg4jU18AjLCKBhRSm],False,6,1,Music & Me,[{'external_urls': {'spotify': 'https://open.s...,...,0,-10.920,1,0.0524,0.2480,0.000246,0.0932,0.897,94.041,3
296,Johnny Raven,213893,False,[Michael Jackson],[spotify:artist:3fMbdgg4jU18AjLCKBhRSm],False,7,1,Music & Me,[{'external_urls': {'spotify': 'https://open.s...,...,10,-10.498,1,0.0295,0.4260,0.000354,0.1180,0.877,111.809,4
297,Euphoria,170666,False,[Michael Jackson],[spotify:artist:3fMbdgg4jU18AjLCKBhRSm],False,8,1,Music & Me,[{'external_urls': {'spotify': 'https://open.s...,...,9,-11.202,0,0.1170,0.1540,0.004140,0.1510,0.876,151.037,4
298,Morning Glow,217973,False,[Michael Jackson],[spotify:artist:3fMbdgg4jU18AjLCKBhRSm],False,9,1,Music & Me,[{'external_urls': {'spotify': 'https://open.s...,...,8,-11.723,1,0.0244,0.2250,0.000352,0.1070,0.744,101.105,4
